In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import ast
import matplotlib.pyplot as plt
import random
from datetime import datetime as dt
from random import randint
from collections import Counter
import math
import sklearn as sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
import string
from nltk.corpus import stopwords
from nltk.stem.porter import *
from sklearn.metrics.pairwise import linear_kernel
from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import roc_curve, auc
from random import randint
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline  

In [ ]:
users = pd.read_csv('users.csv', encoding = "ISO-8859-1")
articles = pd.read_csv('articles.csv', encoding = "ISO-8859-1")

In [ ]:
# take interactions that have sharerHashes
forgraph_t = articles[articles['sharerHash'] != '[]']
forgraph_t = forgraph_t.reset_index()

In [ ]:
# take interactions that have userHashes that are in the users table
users_list = set(users["userHash"])
forgraph = forgraph_t[forgraph_t["userHash"].isin(users_list)] 

In [ ]:
def getRandomInteraction(all_interactions):
    index = randint(0, len(all_interactions)-1)
    currUserHash = all_interactions.iloc[[index]]['userHash'][index]
    currSharerHash_list = ast.literal_eval(all_interactions.iloc[[index]]['sharerHash'][index])
    currSeenTime_list = ast.literal_eval(all_interactions.iloc[[index]]['articleSeenTime'][index])
    if len(currSharerHash_list) == 1:
        currSharerHash = currSharerHash_list[0]
        currSeenTime = currSeenTime_list[0]
    else:
        t_index = randint(0, len(currSharerHash_list)-1)
        currSharerHash = currSharerHash_list[t_index]
        currSeenTime= currSeenTime_list[t_index]
    return [str(currUserHash), str(currSharerHash), str(currSeenTime)]

def makeGraph(timestamp, graph_data):
    timestamp = dt.strptime(timestamp, "%a, %d %b %Y %H:%M:%S %Z")
    beforetime_DF = pd.DataFrame()
    for i in range(0, len(graph_data)):
        temp_date_list = ast.literal_eval(graph_data.iloc[[i]]["articleSeenTime"][i])
        for k in range(0, len(temp_date_list)):
            temp_date = temp_date_list[k]
            date_obj = dt.strptime(temp_date, "%a, %d %b %Y %H:%M:%S %Z")
            if date_obj <= timestamp:
                beforetime_DF = beforetime_DF.append(graph_data.iloc[[i]], ignore_index=True)
                break
                
    sharerHash_list = beforetime_DF['sharerHash']
    userHash_list = beforetime_DF["userHash"]
    
    edges_list = []
    for i in range(0, len(sharerHash_list)):
        temp = ast.literal_eval(sharerHash_list[i])
        for k in range(0, len(temp)):
            edges_list.append((str(userHash_list[i]), temp[k]))
            
    weighted_edges = []
    i = 0
    while i < len(edges_list)-2:
        weight = 1
        while ((edges_list[i][0] == edges_list[i+1][0]) and (edges_list[i][1] == edges_list[i+1][1])):
            weight = weight + 1
            i = i + 1
        weighted_edges.append((edges_list[i][0], edges_list[i][1], weight))
        i = i + 1
        
    interactions = nx.DiGraph()
    interactions.add_weighted_edges_from(weighted_edges)
    return interactions

In [ ]:
def thematicInterest(rand_userHash, users, rand_user_article):
    # get article data
    article_data = getArticleData(rand_user_article)
    #article_counts = Counter(stem(removeStopwords(tokenize(article_data))))
    #print (article_data)
    
    u_data = getUserData(rand_userHash, users)
    #user_counts = Counter(stem(removeStopwords(tokenize(u_data))))
    #print (u_data)
    
    docs = [article_data, u_data]
    tfidf = TfidfVectorizer(stop_words='english')
    output = tfidf.fit_transform(docs)
    cosine_similarities = linear_kernel(output[0:1], output).flatten()
    tfidf = TfidfVectorizer(stop_words='english')
    output = tfidf.fit_transform(docs)
    cosine_similarities = linear_kernel(output[0:1], output).flatten()
    return cosine_similarities[1]

def activity(x, article_data):
    this_user = article_data[article_data["userHash"] == int(x)]
    indeg = len(this_user.index) + 0.01
    shared_by_user = article_data[article_data["sharerHash"] == int(x)]
    outdeg = len(shared_by_user.index) + 0.01
    return outdeg/indeg

def socialPressure(x, article_data, rand_user_article):
    sharers = list(set(article_data[article_data["userHash"] == int(x)]["sharerHash"]))
    R = len(sharers)
    counter = 0
    for index,row in article_data.iterrows():
        if row["sharerHash"] in sharers and row["articleURL"] == rand_user_article["articleURL"]:
            counter = counter + 1
    return counter/R

In [ ]:
def getArticleData(rand_article):
    comments = ast.literal_eval(rand_article["articleComments"])[0]
    title = rand_article["articleTitle"]
    if (comments == "NULL"):
        comments = ""
    if (title == "NULL"):
        title = ""
    article_data = comments + " " + title
    return article_data

def getUserData(userHash, users): #user_info is a df that is a single row
    user_info = users[users["userHash"] == userHash].reset_index()
    u_data = ""
    if str(user_info["currentLocation"][0]) != "nan":
        u_data = u_data + str(user_info["currentLocation"][0]) + " "
    if str(user_info["education1location"][0]) != "nan":
        u_data = u_data + str(user_info["education1location"][0]) + " "
    if str(user_info["education1name"][0]) != "nan":
        u_data = u_data + str(user_info["education1name"][0]) + " "
    if str(user_info["education2location"][0]) != "NaN":
        u_data = u_data + str(user_info["education2location"][0]) + " "
    if str(user_info["education2name"][0]) != "NaN":
        u_data = u_data + str(user_info["education2name"][0]) + " "
    if str(user_info["education3location"][0]) != "NaN":
        u_data = u_data + str(user_info["education3location"][0]) + " "
    if str(user_info["education3name"][0]) != "NaN":
        u_data = u_data + str(user_info["education3name"][0]) + " "
    if str(user_info["gender"][0]) != "NaN":
        u_data = u_data + str(user_info["gender"][0]) + " "
    if str(user_info["hometown"][0]) != "NaN":
        u_data = u_data + str(user_info["hometown"][0]) + " "
    if str(user_info["language1"][0]) != "nan":
        u_data = u_data + str(user_info["language1"][0]) + " "
    if str(user_info["language2"][0]) != "nan":
        u_data = u_data + str(user_info["language2"][0]) + " "
    if str(user_info["language3"][0]) != "nan":
        u_data = u_data + str(user_info["language3"][0]) + " "
    if str(user_info["otherLocation1"][0]) != "nan":
        u_data = u_data + str(user_info["otherLocation1"][0]) + " "
    if str(user_info["otherLocation2"][0]) != "nan":
        u_data = u_data + str(user_info["otherLocation2"][0]) + " "
    if str(user_info["work1location"][0]) != "nan":
        u_data = u_data + str(user_info["work1location"][0]) + " "
    if str(user_info["work1name"][0]) != "nan":
        u_data = u_data + str(user_info["work1name"][0]) + " "
    if str(user_info["work1role"][0]) != "nan":
        u_data = u_data + str(user_info["work1role"][0]) + " "
    if str(user_info["work2location"][0]) != "nan":
        u_data = u_data + str(user_info["work2location"][0]) + " "
    if str(user_info["work2name"][0]) != "nan":
        u_data = u_data + str(user_info["work2name"][0]) + " "
    if str(user_info["work2role"][0]) != "nan":
        u_data = u_data + str(user_info["work2role"][0]) + " "
    if str(user_info["work3location"][0]) != "nan":
        u_data = u_data + str(user_info["work3location"][0]) + " "
    if str(user_info["work3name"][0]) != "nan":
        u_data = u_data + str(user_info["work3name"][0]) + " "
    if str(user_info["work3role"][0]) != "nan":
        u_data = u_data + str(user_info["work3role"][0]) + " "
    if str(user_info["work4location"][0]) != "nan":
        u_data = u_data + str(user_info["work4location"][0]) + " "
    if str(user_info["work4name"][0]) != "nan":
        u_data = u_data + str(user_info["work4name"][0]) + " "
    if str(user_info["work4role"][0]) != "nan":
        u_data = u_data + str(user_info["work4role"][0]) + " "
    return u_data

def getUserArticleSim(article_data, u_data):
    docs = [article_data, u_data]
    tfidf = TfidfVectorizer(stop_words='english')
    output = tfidf.fit_transform(docs)
    cosine_similarities = linear_kernel(output[0:1], output).flatten()
    return cosine_similarities

In [ ]:
features = []

In [ ]:
users_userHashes = users["userHash"]
articles_userHashes = forgraph["userHash"]
valid_userHashes = list(set(users_userHashes) & set(articles_userHashes))

valid_interactions = forgraph[forgraph["userHash"].isin(valid_userHashes)]

count = 0
while len(features) < 1000:
    print(count)
    random_userHash = random.choice(valid_userHashes)

    user_articles = valid_interactions[valid_interactions['userHash'] == random_userHash]
    user_articles_list = user_articles["articleURL"]
    
    random_valid_article = user_articles.sample(n=1).reset_index().iloc(0)
    random_valid_article = random_valid_article[0]
    
    timestamp = ast.literal_eval(random_valid_article["articleSeenTime"])[0]
    timestamp = dt.strptime(timestamp, "%a, %d %b %Y %H:%M:%S %Z")
    beforetime_DF = pd.DataFrame()
    for index, row in valid_interactions.iterrows():
        for i in ast.literal_eval(row["articleSeenTime"]):
            date_obj = dt.strptime(i, "%a, %d %b %Y %H:%M:%S %Z")
            if date_obj <= timestamp:
                beforetime_DF = beforetime_DF.append(row, ignore_index=True)
                break
                
    timestamp_n = getExpectedTime(random_userHash, valid_interactions)
    beforetime_DF_n = pd.DataFrame()
    for index, row in valid_interactions.iterrows():
        for i in ast.literal_eval(row["articleSeenTime"]):
            date_obj = dt.strptime(i, "%a, %d %b %Y %H:%M:%S %Z")
            if date_obj <= timestamp_n:
                beforetime_DF_n = beforetime_DF_n.append(row, ignore_index=True)
                break
    
    non_user_articles = beforetime_DF[beforetime_DF['userHash'] != random_userHash]
    random_notseen_article = non_user_articles.sample(n=1).reset_index().iloc(0)
    random_notseen_article = random_notseen_article[0]
    while random_notseen_article["articleURL"] in user_articles_list:
        random_notseen_article = non_user_articles.sample(n=1).reset_index().iloc(0)
        random_notseen_article = random_notseen_article[0]
    
    s_t = thematicInterest(random_userHash, users, random_valid_article)
    s_f = thematicInterest(random_userHash, users, random_notseen_article)
    a_t = activity(str(random_userHash), beforetime_DF)
    a_f = activity(str(random_userHash), beforetime_DF_n)
    p_t = socialPressure(str(random_userHash), beforetime_DF, random_valid_article)
    p_f = socialPressure(str(random_userHash), beforetime_DF_n, random_notseen_article)
    

    features.append([s_t, a_t, p_t, 1])
    features.append([s_f, a_f, p_f, 0])
    count = count + 1

In [ ]:
X = np.asarray(features)[:,:3]
y = np.asarray(features)[:,3]

In [ ]:
ridge_reg = Ridge(fit_intercept=False)
y_o, predicted_ridge_reg = ridge_reg.fit(X, y, cross_val=True)
ridge_reg.coef_

In [ ]:
fpr_p, tpr_p, thresholds_p = roc_curve(y_o, predicted_ridge_reg)
roc_auc_p = auc(fpr_p, tpr_p)
print ("Area under the PA ROC curve : %f" % roc_auc_p)
matplotlib.rcParams['figure.figsize'] = (10, 10)
plt.plot(fpr_p, tpr_p, color='blue', label='Diffusion Model ROC curve (area = %0.2f)' % roc_auc_p)
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Diffusion Model')
plt.legend(loc="lower right")
plt.savefig('diffusion_model_1000samples_lreg.png')
plt.show()